<a href="https://colab.research.google.com/github/crd3cc04/DS-Unit-2-Linear-Models/blob/master/Copy_of_LS_DS_212_module2-regression-2/_Cortesha_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [12]:
df

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,2,2016-06-02 05:41:05,"30TH/3RD, MASSIVE CONV 2BR IN LUXURY FULL SERV...",E 30 St,40.7426,-73.9790,3200,230 E 30 St,medium,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
49348,1.0,1,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,40.7102,-74.0163,3950,225 Rector Place,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
49349,1.0,1,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49350,1.0,0,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,40.7066,-74.0101,3350,37 Wall Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
# My train/test split data shape

train = df[(df['created'] > '2016-04-01') & (df['created'] < '2016-05-01')]
test = df[(df['created'] > '2016-06-01')]
print(train.shape, test.shape)

(16217, 34) (16973, 34)


In [30]:
# Beginning the baselines
train['price'].mean()

3546.0001849910586

In [0]:
target = 'price'
y_train = train[target]
y_test = test[target]

In [32]:
print('Mean Baseline (using 0 features)')
guess = y_train.mean()

Mean Baseline (using 0 features)


In [33]:
[guess] * len(y_train)

[3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849910586,
 3546.0001849

In [36]:
from sklearn.metrics import mean_absolute_error

y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error (Price of Apartments in April & May 2016): {mae:.4f} percentage points')

Train Error (Price of Apartments in April & May 2016): 1176.0141 percentage points


In [35]:
y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error (Price of Apartments in June 2016): {mae:.4f} percentage points')

Test Error (Price of Apartments in June 2016): 1189.6597 percentage points


In [47]:
import plotly.express as px

px.scatter(
    train, 
    x ='bathrooms',
    y ='bedrooms',
    text ='price',
    title ='Apartment Rental Prices in New York City',
    trendline='ols',
)

In [0]:
from sklearn.linear_model import LinearRegression

In [0]:
model = LinearRegression()

In [57]:
# Arranging X features matrices
features = ['bedrooms', 'bathrooms']
print(f'Linear Regression, dependent on: {features}')
X_train = train[features]
X_test = test[features]
print(X_train.shape, X_test.shape)

Linear Regression, dependent on: ['bedrooms', 'bathrooms']
(16217, 2) (16973, 2)


In [58]:
# Fitting the model 
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: {mae:.2f} percentage points')

Train Error: 814.71 percentage points


In [59]:
# Applying the model to the new data
y_test_pred =  model.predict(X_test)
mae_test = mean_absolute_error(y_test, y_test_pred)
print(f'Test Error: {mae_test:.2f} percentage points')

Test Error: 824.07 percentage points


In [60]:
# Getting the Coefficients and Intercept
model.intercept_, model.coef_

(562.5200052673877, array([ 393.47218465, 1991.89599453]))

In [63]:
# Intercept and Coefficients
print('Intercept', model.intercept_)
coeffiecients = pd.Series(model.coef_, features)
print(coeffiecients.to_string())

Intercept 562.5200052673877
bedrooms      393.472185
bathrooms    1991.895995


In [0]:
#Creating a function for getting the RMSE, MAE, and  R2

import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def squared_errors(df, features, target, z, b):

  x = df[features]
  y = df[target]
  y_pred = z*x + b

  mse = mean_squared_error(y, y_pred)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(y, y_pred)
  r2 = r2_score(y, y_pred)
  print('Mean Squared Error:', mse)
  print('Root Mean Squared Error:', rmse)
  print('Mean Absolute Error:', mae)
  print('R^2:', r2)


In [72]:
# RMSE, MAE, and  R2 for my Train data
feature = ['bedrooms']
squared_errors(train, feature, target, z=0, b=y_train.mean())

Mean Squared Error: 2983356.8650798206
Root Mean Squared Error: 1727.239666369384
Mean Absolute Error: 1176.0140505064794
R^2: 0.0


In [73]:
# RMSE, MAE, and  R2 for my Test data
feature1 = ['bedrooms']
squared_errors(test, feature1, target, z=0, b=y_test.mean())

Mean Squared Error: 3108021.267852562
Root Mean Squared Error: 1762.9581015590138
Mean Absolute Error: 1200.8695333832386
R^2: 0.0
